# MLAOIV: Many Weak Instruments

This notebook demonstrates Machine Learning Approximate Optimal Instrumental Variables (MLAOIV) in a many-weak-instruments setting.

## Model

We consider a standard IV setup with $d=500$ instruments:
- **First stage**: $y_1 = \alpha + \pi' Z + u$
- **Structural equation**: $y_2 = \gamma + \beta y_1 + e$

where $(u, e)$ are correlated (ρ=0.5), creating endogeneity.

## MLAOIV Approach

Instead of using all instruments directly, we construct the optimal IV as $\hat{E}[y_1 | Z]$ using ML methods with cross-validation.

In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import cross_val_predict
from sklearn.linear_model import LassoCV, RidgeCV, ElasticNetCV, LinearRegression
from linearmodels.iv import IV2SLS
from joblib import Parallel, delayed
import warnings
warnings.filterwarnings("ignore")

np.random.seed(42)

In [2]:
def simulate_iv_data(n_obs, n_instruments, error_covariance=None, 
                      instrument_strength=0.5, true_beta=0.75, 
                      true_intercept=-0.9, seed=None):
    """
    Simulate data for IV regression.
    
    Model:
        y1 = 0.3 + pi * Z + u  (first stage)
        y2 = intercept + beta * y1 + e  (structural)
        where (u, e) ~ N(0, error_covariance)
    """
    if seed is not None:
        np.random.seed(seed)
    if error_covariance is None:
        error_covariance = np.array([[1, 0.5], [0.5, 1]])
    
    # Generate errors
    errors = np.random.multivariate_normal([0, 0], error_covariance, n_obs)
    u, e = errors[:, 0], errors[:, 1]
    
    # Generate instruments
    Z = np.random.normal(0, 1, (n_obs, n_instruments))
    
    # All instruments are relevant with equal weight
    pi = np.ones(n_instruments)
    scale = (50 / n_instruments) * instrument_strength
    
    # Generate variables
    y1 = 0.3 + scale * (Z @ pi) + u
    y2 = true_intercept + true_beta * y1 + e
    
    return {'Z': Z, 'y1': y1, 'y2': y2}


def compute_mlaoiv(Z, y_endog, regressor, cv_folds=3):
    """
    Compute MLAOIV using cross-validated predictions.
    
    Parameters:
        Z: instrument matrix (n x d)
        y_endog: endogenous variable
        regressor: sklearn estimator (e.g., RidgeCV, LassoCV)
        cv_folds: number of CV folds for prediction
    """
    return cross_val_predict(regressor, Z, y_endog, cv=cv_folds)


def estimate_iv2sls(y_outcome, y_endog, mlaoiv_instrument):
    """Estimate IV-2SLS using MLAOIV instrument."""
    df = pd.DataFrame({'y2': y_outcome, 'y1': y_endog, 'mlaoiv': mlaoiv_instrument})
    model = IV2SLS.from_formula("y2 ~ 1 + [y1 ~ mlaoiv]", data=df).fit()
    return {'params': np.array(model.params), 'std_errors': np.array(model.std_errors), 'model': model}

## 1. Single Simulation Example

In [3]:
# Simulation parameters
n_obs = 1000
n_instruments = 500
error_covariance = np.array([[1, 0.5], [0.5, 1]])

# True parameters
true_beta = 0.75
true_intercept = -0.9

print(f"Sample size: {n_obs}")
print(f"Number of instruments: {n_instruments}")
print(f"True beta: {true_beta}")
print(f"True intercept: {true_intercept}")

Sample size: 1000
Number of instruments: 500
True beta: 0.75
True intercept: -0.9


In [4]:
# Generate data
data = simulate_iv_data(
    n_obs=n_obs,
    n_instruments=n_instruments,
    error_covariance=error_covariance,
    instrument_strength=0.5,
    seed=42
)

print(f"Z shape: {data['Z'].shape}")
print(f"y1 shape: {data['y1'].shape}")
print(f"y2 shape: {data['y2'].shape}")

Z shape: (1000, 500)
y1 shape: (1000,)
y2 shape: (1000,)


In [5]:
# Compute MLAOIV using Ridge regression
ridge = RidgeCV(cv=4, alphas=[2000, 1000, 100, 50, 10, 1, 0.1])

mlaoiv = compute_mlaoiv(
    Z=data['Z'],
    y_endog=data['y1'],
    regressor=ridge,
    cv_folds=3
)

print(f"MLAOIV instrument computed. Shape: {mlaoiv.shape}")
print(f"Correlation with y1: {np.corrcoef(mlaoiv, data['y1'])[0,1]:.4f}")

MLAOIV instrument computed. Shape: (1000,)
Correlation with y1: 0.5002


In [6]:
# Estimate IV-2SLS
results = estimate_iv2sls(
    y_outcome=data['y2'],
    y_endog=data['y1'],
    mlaoiv_instrument=mlaoiv
)

print("\n=== MLAOIV Estimation Results ===")
print(results['model'].summary)


=== MLAOIV Estimation Results ===
                          IV-2SLS Estimation Summary                          
Dep. Variable:                     y2   R-squared:                      0.6356
Estimator:                    IV-2SLS   Adj. R-squared:                 0.6353
No. Observations:                1000   F-statistic:                    260.99
Date:                Sun, Jan 18 2026   P-value (F-stat)                0.0000
Time:                        23:10:45   Distribution:                  chi2(1)
Cov. Estimator:                robust                                         
                                                                              
                             Parameter Estimates                              
            Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
------------------------------------------------------------------------------
Intercept     -0.8915     0.0334    -26.707     0.0000     -0.9569     -0.8261
y1             0.

In [7]:
# OLS (biased due to endogeneity)
X_ols = np.column_stack([np.ones(n_obs), data['y1']])
ols_beta = np.linalg.lstsq(X_ols, data['y2'], rcond=None)[0]

print("=== OLS vs MLAOIV ===")
print(f"{'Method':<12} {'Intercept':<12} {'Beta':<12} {'Beta Bias':<12}")
print("-" * 48)
print(f"{'OLS':<12} {ols_beta[0]:<12.4f} {ols_beta[1]:<12.4f} {ols_beta[1] - true_beta:<12.4f}")
print(f"{'MLAOIV':<12} {results['params'][0]:<12.4f} {results['params'][1]:<12.4f} {results['params'][1] - true_beta:<12.4f}")
print(f"{'True':<12} {true_intercept:<12.4f} {true_beta:<12.4f} {0.0:<12.4f}")

=== OLS vs MLAOIV ===
Method       Intercept    Beta         Beta Bias   
------------------------------------------------
OLS          -0.9410      0.9481       0.1981      
MLAOIV       -0.8915      0.7073       -0.0427     
True         -0.9000      0.7500       0.0000      


## 2. Compare Different ML Methods

In [8]:
# Compare Lasso, Ridge, and ElasticNet
regressors = {
    'Lasso': LassoCV(cv=4, alphas=[2000, 1000, 100, 50, 10, 1, 0.1], max_iter=10000),
    'Ridge': RidgeCV(cv=4, alphas=[2000, 1000, 100, 50, 10, 1, 0.1]),
    'ElasticNet': ElasticNetCV(cv=4, l1_ratio=[0.1, 0.5, 0.9], 
                               alphas=[2000, 1000, 100, 50, 10], max_iter=10000)
}

results_list = []
for name, regr in regressors.items():
    mlaoiv = compute_mlaoiv(data['Z'], data['y1'], regressor=regr, cv_folds=3)
    est = estimate_iv2sls(data['y2'], data['y1'], mlaoiv)
    results_list.append({
        'Method': name,
        'Beta Est': est['params'][1],
        'Beta SE': est['std_errors'][1],
        'Beta Bias': est['params'][1] - true_beta
    })

comparison = pd.DataFrame(results_list)
print("\n=== Method Comparison ===")
print(comparison.to_string(index=False))


=== Method Comparison ===
    Method  Beta Est  Beta SE  Beta Bias
     Lasso  0.455085 0.210854  -0.294915
     Ridge  0.707349 0.043785  -0.042651
ElasticNet  0.830271 0.343409   0.080271


## 3. Monte Carlo Simulation

In [9]:
def single_simulation(seed):
    """Run a single simulation and return results."""
    data = simulate_iv_data(
        n_obs=1000,
        n_instruments=500,
        error_covariance=np.array([[1, 0.5], [0.5, 1]]),
        instrument_strength=0.5,
        seed=seed
    )
    ridge = RidgeCV(cv=4, alphas=[2000, 1000, 100, 50, 10, 1, 0.1])
    mlaoiv = compute_mlaoiv(data['Z'], data['y1'], regressor=ridge, cv_folds=3)
    results = estimate_iv2sls(data['y2'], data['y1'], mlaoiv)
    return {'params': results['params'], 'std_errors': results['std_errors']}

# Run 20 simulations in parallel
n_sims = 20
print(f"Running {n_sims} Monte Carlo simulations...")
mc_results = Parallel(n_jobs=-1)(
    delayed(single_simulation)(seed) for seed in range(n_sims)
)

# Aggregate results
params_matrix = np.array([r['params'] for r in mc_results])
se_matrix = np.array([r['std_errors'] for r in mc_results])

true_params = np.array([true_intercept, true_beta])

bias = np.mean(params_matrix - true_params, axis=0)
avg_se = np.mean(se_matrix, axis=0)
rmse = np.sqrt(np.mean((params_matrix - true_params)**2, axis=0))

print("\n=== Monte Carlo Results (Ridge MLAOIV) ===")
print(f"{'Parameter':<12} {'Bias':<12} {'Avg SE':<12} {'RMSE':<12}")
print("-" * 48)
print(f"{'Intercept':<12} {bias[0]:<12.4f} {avg_se[0]:<12.4f} {rmse[0]:<12.4f}")
print(f"{'Beta':<12} {bias[1]:<12.4f} {avg_se[1]:<12.4f} {rmse[1]:<12.4f}")

Running 20 Monte Carlo simulations...

=== Monte Carlo Results (Ridge MLAOIV) ===
Parameter    Bias         Avg SE       RMSE        
------------------------------------------------
Intercept    -0.0038      0.0338       0.0356      
Beta         0.0111       0.0407       0.0451      
